In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
%%capture
!pip install datasets transformers huggingface_hub torchaudio librosa jiwer
!pip install accelerate -U
!pip install nvidia-smi
!pip install transformers[torch]
!pip install wandb
!pip install matplotlib seaborn
!pip install evaluate
import os
import re
import wandb
import torch
import numpy as np
import json
from evaluate import load
from datasets import load_dataset, Audio
from transformers import EarlyStoppingCallback
from huggingface_hub import HfFolder


In [ ]:
from datasets import load_dataset, DatasetDict

test5hr = load_dataset("KYAGABA/oromo_cleaned_dataset")['test']
test5hr = test5hr.rename_column('text', 'transcription')

print(test5hr)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/640 [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/21 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/21 [00:00<?, ?it/s]

train-00000-of-00021.parquet:   0%|          | 0.00/420M [00:00<?, ?B/s]

...

Generating train split:   0%|          | 0/46865 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2246 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1103 [00:00<?, ? examples/s]

Loading dataset shards:   0%|          | 0/20 [00:00<?, ?it/s]

Dataset({
    features: ['audio', 'transcription', 'speaker', 'split'],
    num_rows: 1103
})


In [ ]:
import unicodedata
import re
def preprocess_text(batch):
    batch['transcription'] = [text.lower() for text in batch['transcription']]
    batch['transcription'] = [unicodedata.normalize('NFKC', text) for text in batch['transcription']]
    batch['transcription'] = [re.sub(r'[\’\ʻ\ʼ\ʽ\‘\']', '', text) for text in batch['transcription']]
    batch['transcription'] = [re.sub(r'[^\w\s\']', '', text) for text in batch['transcription']]
    batch['transcription'] = [re.sub(r'_', ' ', text) for text in batch['transcription']]
    batch['transcription'] = [' '.join(text.split()) for text in batch['transcription']]
    return batch

test5hr = test5hr.map(preprocess_text, batched=True)

Map:   0%|          | 0/1103 [00:00<?, ? examples/s]

In [ ]:
def extract_all_chars(batch):
    all_text = " ".join(batch["transcription"])
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}
vocab_test = test5hr.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=test5hr.column_names)
vocab_list = list(set(vocab_test["vocab"][0]))
vocab_dict = {v: k for k, v in enumerate(sorted(vocab_list))}
vocab_dict

Map:   0%|          | 0/1103 [00:00<?, ? examples/s]

{' ': 0,
 'a': 1,
 'b': 2,
 'c': 3,
 'd': 4,
 'e': 5,
 'f': 6,
 'g': 7,
 'h': 8,
 'i': 9,
 'j': 10,
 'k': 11,
 'l': 12,
 'm': 13,
 'n': 14,
 'o': 15,
 'p': 16,
 'q': 17,
 'r': 18,
 's': 19,
 't': 20,
 'u': 21,
 'v': 22,
 'w': 23,
 'x': 24,
 'y': 25,
 'z': 26}

In [ ]:
from datasets import load_dataset, DatasetDict
import torch
from transformers.pipelines.pt_utils import KeyDataset
from transformers import pipeline
from tqdm import tqdm
import evaluate
import pandas as pd

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

models= [
    "asr-africa/facebook-mms-1b-all-Sagalee-orm-1hrs-v1",
    "asr-africa/mms-1b-all-Sagalee-orm-5hrs-1",
    "asr-africa/mms-1b-all-Sagalee-orm-10hrs-1",
    ""

]
# Evaluation metrics
wer_metric = evaluate.load("wer")
cer_metric = evaluate.load("cer")

def compute_individual_metrics(prediction, reference):
    wer = wer_metric.compute(predictions=[prediction], references=[reference])
    cer = cer_metric.compute(predictions=[prediction], references=[reference])
    return wer, cer

def run_inference_for_model(model_name):
    print(f"Processing model: {model_name}")
    pipe = pipeline("automatic-speech-recognition", model=model_name, device=device)
    results = []
    all_predictions = []
    all_references = []
    for audio, transcription in tqdm(zip(test5hr["audio"], test5hr["transcription"]),                                                          total=test5hr.num_rows, desc="Running inference"):
        prediction = pipe(audio)['text']
        wer, cer = compute_individual_metrics(prediction, transcription)

        results.append({
            "Prediction": prediction,
            "Reference": transcription,
            "WER": wer,
            "CER": cer,
        })
        all_predictions.append(prediction)
        all_references.append(transcription)

    df = pd.DataFrame(results)
    overall_wer = wer_metric.compute(predictions=all_predictions, references=all_references)
    overall_cer = cer_metric.compute(predictions=all_predictions, references=all_references)

    print("Calculated overall WER:", overall_wer)
    print("Calculated overall CER:", overall_cer)
    # average_metrics_by_source = df.groupby("Source").agg({"WER": "mean", "CER": "mean"}).reset_index()
    # print("\nAverage WER and CER by Source:")
    # print(average_metrics_by_source)
    # Save detailed results to CSV
    output_filename = f'transcriptions_{model_name.split("/")[-1]}.csv'
    # output_filename = f"{model_name}_detailed_results.csv"
    df.to_csv(output_filename, index=False)
    print(f"Results saved to {output_filename}")
# Loop through each model and evaluate
for model in models:
    run_inference_for_model(model)

Processing model: asr-africa/facebook-mms-1b-all-Sagalee-orm-1hrs-v1


config.json:   0%|          | 0.00/2.05k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/320 [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/320 [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/30.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/256 [00:00<?, ?B/s]

Running inference:  48%|████▊     | 533/1103 [2:29:51<2:08:16, 13.50s/it]

In [ ]:
from datasets import load_dataset, DatasetDict
import torch
from transformers.pipelines.pt_utils import KeyDataset
from transformers import pipeline
from tqdm import tqdm
import evaluate
import pandas as pd

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

models= [
    "asr-africa/wav2vec2-large-common_voice-rw-100hrs-v2"
]
# Evaluation metrics
wer_metric = evaluate.load("wer")
cer_metric = evaluate.load("cer")

def compute_individual_metrics(prediction, reference):
    wer = wer_metric.compute(predictions=[prediction], references=[reference])
    cer = cer_metric.compute(predictions=[prediction], references=[reference])
    return wer, cer

def run_inference_for_model(model_name):
    print(f"Processing model: {model_name}")
    pipe = pipeline("automatic-speech-recognition", model=model_name, device=device)
    results = []
    all_predictions = []
    all_references = []

    for audio, transcription in tqdm(zip(test5hr["audio"], test5hr["transcription"]),
                                                         total=test5hr.num_rows, desc="Running inference"):
        prediction = pipe(audio)['text']
        wer, cer = compute_individual_metrics(prediction, transcription)

        results.append({
            "Prediction": prediction,
            "Reference": transcription,
            "WER": wer,
            "CER": cer,
        })
        all_predictions.append(prediction)
        all_references.append(transcription)

    df = pd.DataFrame(results)
    overall_wer = wer_metric.compute(predictions=all_predictions, references=all_references)
    overall_cer = cer_metric.compute(predictions=all_predictions, references=all_references)

    print("Calculated overall WER:", overall_wer)
    print("Calculated overall CER:", overall_cer)

    # average_metrics_by_source = df.groupby("Source").agg({"WER": "mean", "CER": "mean"}).reset_index()
    # print("\nAverage WER and CER by Source:")
    # print(average_metrics_by_source)

    # Save detailed results to CSV
    output_filename = f'transcriptions_{model_name.split("/")[-1]}.csv'
    # output_filename = f"{model_name}_detailed_results.csv"
    df.to_csv(output_filename, index=False)
    print(f"Results saved to {output_filename}")

# Loop through each model and evaluate
for model in models:
    run_inference_for_model(model)


Processing model: asr-africa/wav2vec2-large-common_voice-rw-100hrs-v2


Running inference: 100%|██████████| 4981/4981 [02:06<00:00, 39.52it/s]


Calculated overall WER: 1.0
Calculated overall CER: 2.9760707227304466
Results saved to transcriptions_wav2vec2-large-common_voice-rw-100hrs-v2.csv
